In [2]:
#pip install requests # HTTP 요청을 보내는 모듈 설치

In [3]:
# pip install bs4 # HTML 코드 파싱 모듈 설치

In [4]:
# pip install pandas # 데이터 분석 및 처리 모듈 설치

In [54]:
# pip install pymysql # mysql 라이브러리 설치

Note: you may need to restart the kernel to use updated packages.


In [39]:
import requests # request 모듈 호출
import pandas as pd # 데이터 처리 및 분석 모듈 호출
from bs4 import BeautifulSoup # bs4 모듈 호출
import datetime as dt

In [48]:
title = [] # 기사 제목 배열
news_url = [] # 기사 url 배열
press = [] # 신문사
wtime = [] # 기사 작성일

first_url = 'https://finance.naver.com/news/news_list.naver?mode=LSS3D&section_id=101&section_id2=258&section_id3=403' # 해외증시 첫 페이지 url
first_resp = requests.get(first_url) # 해외증시 첫 페이지 GET 방식 크롤링
first_html = first_resp.text # readable content 가져오기
first_soup = BeautifulSoup(first_html, 'html.parser') # html 코드 형태로 구분
pgRR = first_soup.select('.pgRR') # 전체 페이지가 나오는 클래스
page_atag = pgRR[0].find('a')['href'] # a태그 href url 가져옴
index = page_atag.find("page=") # href url에 page 파라미터 값 가져옴
page = int(page_atag[index + len("page="):])

In [49]:
day = dt.datetime.now()
today = day.strftime("%Y%m%d")
for page_num in range(1, page+1):
    url = 'https://finance.naver.com/news/news_list.naver?mode=LSS3D&section_id=101&section_id2=258&section_id3=403&date=' + today + '&page=' + str(page_num)
    resp = requests.get(url) # GET 방식 크롤링
    html = resp.text
    soup = BeautifulSoup(html, 'html.parser') # HTML 코드 형태로 구분
    news = soup.select('.articleSubject') # 페이지에 있는 뉴스 가져오기
    summary = soup.select('.articleSummary') # 기사 별 날짜 가져오기
    for i in news:
        title.append(i.text.strip('\n')) # 개행문자 치환하여 배열에 추가
        httpstr = i.find('a')['href'] # a태그의 href url을 가져옴
        start = httpstr.find("§ion_id") # url을 뉴스 주소에 맞게 필요없는 부분 제거
        end = httpstr.find("date") # url을 뉴스 주소에 맞게 필요없는 부분 제거
        modified_string = httpstr[:start] + "&" + httpstr[end:] # url을 뉴스 주소에 맞게 필요없는 부분 제거
        naver_news = "https://finance.naver.com" # 네이버 경제뉴스 url
        all_url = naver_news + modified_string # 기사별 url 완성
        news_url.append(all_url) # 완성된 url을 news_url 배열에 추가
        
    for i in summary:
        press.append(i.find('span', {"class" : "press"}).text)
        wtime.append(i.find('span', {"class" : "wdate"}).text)

In [52]:
df = pd.DataFrame()
df['제목'] = title
df['URL'] = news_url
df['COMPANY'] = press
df['TIME'] = wtime

df

,제목,URL,COMPANY,TIME
0,지난해 미국·캐나다증시 공매도 투자자 1900억달러 손실,https://finance.naver.com/news/news_read.naver...,아시아경제,2024-01-05 20:20
1,美장기채 투자 서학개미 '시련의 계절' 오나,https://finance.naver.com/news/news_read.naver...,매일경제,2024-01-05 17:33
2,北포격·美비농업고용 경계심…환율 1315원으로 상승[외환마감],https://finance.naver.com/news/news_read.naver...,이데일리,2024-01-05 16:28
3,작년 미국·캐나다 증시 공매도 투자자 256조원 손실…'M7' 대거 포함,https://finance.naver.com/news/news_read.naver...,이데일리,2024-01-05 15:23
4,작년 미국·캐나다 증시 공매도 투자자 256조원 손실,https://finance.naver.com/news/news_read.naver...,연합뉴스,2024-01-05 14:41
5,"‘최고의 선택’ 아멕스, 경기 우려 속 안전한 피난처? (영상)",https://finance.naver.com/news/news_read.naver...,이데일리,2024-01-05 13:48
6,"""가격인상 지나치다""...까르푸, 펩시·치토스 등 판매 거부",https://finance.naver.com/news/news_read.naver...,한국경제TV,2024-01-05 13:45
7,"美고용 열기 ‘경계’…장중 환율, 1310원 초반대 횡보[외환분석]",https://finance.naver.com/news/news_read.naver...,이데일리,2024-01-05 12:09
8,"애플 보다 더 오르고 덜 내린 MS...""애저 클라우드 부문 수혜""",https://finance.naver.com/news/news_read.naver...,한국경제TV,2024-01-05 11:04
9,애플 투자의견 또 강등…이러다 ‘시총 1위 왕좌’ 위험할수도?,https://finance.naver.com/news/news_read.naver...,매일경제,2024-01-05 11:04
